In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cp -r /content/drive/MyDrive/recipes_texts.csv /content/
!cp -r /content/drive/MyDrive/recipes_pages_clean.csv /content/

In [4]:
import pandas as pd


pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)


recipes_texts = pd.read_csv('/content/recipes_texts.csv')
print("Recipes Texts (полная первая строка):")
display(recipes_texts.iloc[[0]])


print("\nИнформация о recipes_texts:")
display(recipes_texts.info())




Recipes Texts (полная первая строка):


,link,image_link,title,description,ingredients,recipe
0,https://www.russianfood.com/recipes/recipe.php?rid=123285,https://img1.russianfood.com/dycontent/images_upl/42/big_41966.jpg,"""бефстроганов"" из куриной грудки","фальшивый бефстроганов по мотивам любимого рецепта. меня всегда выручает курятина. быстро готовится, а вкусно как, м-м-м! представляю рецепт бефстроганова из куриной грудки, думаю вам понравится. а с красивым оформлением пойдет и как горячее блюдо на новый год 2025.","['филе куриное (грудка) - 500 г', 'лук репчатый - 1 шт.', 'мука - 1 ст. ложка', 'сливки - 150 мл', 'томатный сок - 50 мл', 'горчица - 1 ч. ложка', 'соль - 1 ч. ложка', 'перец - 1 щепотка', 'растительное масло - 3 ст. ложки']","['продукты для бефстроганова из курицы перед вами.', 'как приготовить бефстроганов из куриной грудки:\n\r\nкуриную грудку вымыть и обсушить.', 'филе куриной грудки разрезать на 2-3 плоских порционных куска и нарезать тонкой соломкой.', 'разогреть сковороду, налить растительное масло. выложить мясо.', 'филе посолить, поперчить и слегка обжарить 5-7 минут, помешивая.', 'лук очистить и помыть, порезать полукольцами.', 'затем к мясу добавить лук и обжарить до подрумянивания лука 4-5 минут, помешивая.', 'добавить муку, хорошо перемешать.', 'все залить сливками.', 'смешать томатный сок с горчицей. хорошо перемешать.', 'затем добавить к мясу горчицу и томатный сок.', 'тушить куриный бефстроганов на слабом огне под крышкой 10-15 минут.', 'бефстроганов из куриной грудки готов. подавать с любимым гарниром.\r\nприятного аппетита!']"



Информация о recipes_texts:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92144 entries, 0 to 92143
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   link         92144 non-null  object
 1   image_link   92059 non-null  object
 2   title        92059 non-null  object
 3   description  49992 non-null  object
 4   ingredients  92144 non-null  object
 5   recipe       92144 non-null  object
dtypes: object(6)
memory usage: 4.2+ MB


None

In [5]:
import pandas as pd
import re
from tqdm.auto import tqdm


pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)


recipes_texts = pd.read_csv('/content/recipes_texts.csv')


print("=== Исходный датасет (первая строка до чистки): ===")
display(recipes_texts.iloc[[0]])
print("\nИнформация о датасете:")
display(recipes_texts.info())


def safe_eval(x):
    if pd.isna(x):
        return []
    if isinstance(x, str):
        try:
            return eval(x)
        except:
            return []
    return x

recipes_texts['ingredients'] = recipes_texts['ingredients'].apply(safe_eval)
recipes_texts['recipe'] = recipes_texts['recipe'].apply(safe_eval)


def find_special_chars(text):
    if pd.isna(text) or not isinstance(text, str):
        return set()

    allowed_chars = re.compile(r'[а-яА-Яa-zA-Z0-9.,!?:"\';() \n\r-]')

    special_chars = set(re.findall(r'[^а-яА-Яa-zA-Z0-9.,!?:"\';() \n\r-]', text))
    return special_chars


special_chars_title = set()
special_chars_description = set()
special_chars_ingredients = set()
special_chars_recipe = set()


for title in recipes_texts['title']:
    special_chars_title.update(find_special_chars(title))

for desc in recipes_texts['description']:
    special_chars_description.update(find_special_chars(desc))


for ingred_list in recipes_texts['ingredients']:
    for ingred in ingred_list:
        special_chars_ingredients.update(find_special_chars(ingred))

for recipe_list in recipes_texts['recipe']:
    for step in recipe_list:
        special_chars_recipe.update(find_special_chars(step))


all_special_chars = special_chars_title.union(special_chars_description,
                                              special_chars_ingredients,
                                              special_chars_recipe)
print("\n=== Уникальные спецсимволы в датасете: ===")
print(all_special_chars)


def find_examples_with_char(df, column, char):
    examples = df[df[column].str.contains(re.escape(char), na=False)][column].tolist()
    return examples[:3]
def find_ingredient_examples(df, char):
    examples = []
    for idx, ingred_list in enumerate(df['ingredients']):
        for ingred in ingred_list:
            if char in ingred:
                examples.append(ingred)
                if len(examples) >= 3:
                    return examples
    return examples

def find_recipe_examples(df, char):
    examples = []
    for idx, recipe_list in enumerate(df['recipe']):
        for step in recipe_list:
            if char in step:
                examples.append(step)
                if len(examples) >= 3:
                    return examples
    return examples


print("\n=== Примеры использования спецсимволов: ===")
for char in all_special_chars:
    print(f"\nСимвол '{char}':")

    if char in special_chars_title:
        examples = find_examples_with_char(recipes_texts, 'title', char)
        print(f"  Примеры в title: {examples}")

    if char in special_chars_description:
        examples = find_examples_with_char(recipes_texts, 'description', char)
        print(f"  Примеры в description: {examples}")

    if char in special_chars_ingredients:
        examples = find_ingredient_examples(recipes_texts, char)
        print(f"  Примеры в ingredients: {examples}")

    if char in special_chars_recipe:
        examples = find_recipe_examples(recipes_texts, char)
        print(f"  Примеры в recipe: {examples}")



=== Исходный датасет (первая строка до чистки): ===


,link,image_link,title,description,ingredients,recipe
0,https://www.russianfood.com/recipes/recipe.php?rid=123285,https://img1.russianfood.com/dycontent/images_upl/42/big_41966.jpg,"""бефстроганов"" из куриной грудки","фальшивый бефстроганов по мотивам любимого рецепта. меня всегда выручает курятина. быстро готовится, а вкусно как, м-м-м! представляю рецепт бефстроганова из куриной грудки, думаю вам понравится. а с красивым оформлением пойдет и как горячее блюдо на новый год 2025.","['филе куриное (грудка) - 500 г', 'лук репчатый - 1 шт.', 'мука - 1 ст. ложка', 'сливки - 150 мл', 'томатный сок - 50 мл', 'горчица - 1 ч. ложка', 'соль - 1 ч. ложка', 'перец - 1 щепотка', 'растительное масло - 3 ст. ложки']","['продукты для бефстроганова из курицы перед вами.', 'как приготовить бефстроганов из куриной грудки:\n\r\nкуриную грудку вымыть и обсушить.', 'филе куриной грудки разрезать на 2-3 плоских порционных куска и нарезать тонкой соломкой.', 'разогреть сковороду, налить растительное масло. выложить мясо.', 'филе посолить, поперчить и слегка обжарить 5-7 минут, помешивая.', 'лук очистить и помыть, порезать полукольцами.', 'затем к мясу добавить лук и обжарить до подрумянивания лука 4-5 минут, помешивая.', 'добавить муку, хорошо перемешать.', 'все залить сливками.', 'смешать томатный сок с горчицей. хорошо перемешать.', 'затем добавить к мясу горчицу и томатный сок.', 'тушить куриный бефстроганов на слабом огне под крышкой 10-15 минут.', 'бефстроганов из куриной грудки готов. подавать с любимым гарниром.\r\nприятного аппетита!']"



Информация о датасете:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92144 entries, 0 to 92143
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   link         92144 non-null  object
 1   image_link   92059 non-null  object
 2   title        92059 non-null  object
 3   description  49992 non-null  object
 4   ingredients  92144 non-null  object
 5   recipe       92144 non-null  object
dtypes: object(6)
memory usage: 4.2+ MB


None


=== Уникальные спецсимволы в датасете: ===
{'ς', '′', 'ә', '±', '🤔', 'è', 'ă', '&', '’', '`', '<', '×', 'á', 'վ', '❤', 'ї', '\t', 'ｻ', 'ş', '¾', '©', '\u200a', '*', '}', 'č', '‒', '³', '>', '=', 'μ', '−', '÷', 'î', '\\', '℃', 'ρ', '！', '\u200d', 'ｫ', '\xa0', '\uf04a', '“', 'ր', 'ա', 'έ', '😋', '„', '¼', '№', 'ν', 'à', '\u2009', ']', '\x9d', 'š', 'γ', '½', 'é', 'â', 'ß', 'ø', 'υ', 'ä', '\ufeff', '—', 'ö', 'є', '🍓', 'і', '）', '«', '~', '•', '”', '\xad', '😍', '»', '˚', '─', '😃', '²', 'љ', '⅖', 'ո', '–', '🔥', '（', '|', '̆', 'ú', '≪', '⅘', 'զ', '․', 'ç', 'φ', 'ý', '^', '¬', '\u2008', '´', 'ť', '≈', 'ã', '{', 'ı', 'ο', '…', 'ι', '\u202f', '⅓', '́', '😉', 'κ', 'ó', '✍', 'α', 'њ', '⅗', 'û', '🌼', 'գ', '%', '\x7f', 'ё', '\u2063', 'λ', '‚', '⅔', '―', '\u2006', '⁰', 'º', 'ï', 'ë', 'ñ', 'í', '+', 'ü', '⠀', '≫', '/', '°', '_', '‑', '[', '\u200b'}

=== Примеры использования спецсимволов: ===

Символ 'ς':
  Примеры в description: ['факес (греч. φακές - чечевица) - это популярный в греции густой суп с ч

In [6]:
chars_to_remove = ['*', '&', '@', '#','•','ｻ','🌼','😉','_','❤','😉','¬','🔥','😋','','ø','😃','😍',"©",'🍓','🤔','✍']  # Замени на актуальный список

print("\n=== Символы, которые будут удалены: ===")
print(chars_to_remove)


def clean_text(text, chars_to_remove):
    if pd.isna(text) or not isinstance(text, str):
        return text
    pattern = '[' + re.escape(''.join(chars_to_remove)) + ']'
    return re.sub(pattern, '', text)

def clean_list(lst, chars_to_remove):
    return [clean_text(item, chars_to_remove) for item in lst]


recipes_texts['title'] = recipes_texts['title'].apply(lambda x: clean_text(x, chars_to_remove))
recipes_texts['description'] = recipes_texts['description'].apply(lambda x: clean_text(x, chars_to_remove))
recipes_texts['ingredients'] = recipes_texts['ingredients'].apply(lambda x: clean_list(x, chars_to_remove))
recipes_texts['recipe'] = recipes_texts['recipe'].apply(lambda x: clean_list(x, chars_to_remove))


print("\n=== Очищенный датасет (первая строка после чистки): ===")
display(recipes_texts.iloc[[0]])


recipes_texts.to_csv('/content/recipes_texts_cleaned.csv', index=False)
print("\nОчищенный датасет сохранен в '/content/recipes_texts_cleaned.csv'")


=== Символы, которые будут удалены: ===
['*', '&', '@', '#', '•', 'ｻ', '🌼', '😉', '_', '❤', '😉', '¬', '🔥', '😋', '\uf04a', 'ø', '😃', '😍', '©', '🍓', '🤔', '✍']

=== Очищенный датасет (первая строка после чистки): ===


,link,image_link,title,description,ingredients,recipe
0,https://www.russianfood.com/recipes/recipe.php?rid=123285,https://img1.russianfood.com/dycontent/images_upl/42/big_41966.jpg,"""бефстроганов"" из куриной грудки","фальшивый бефстроганов по мотивам любимого рецепта. меня всегда выручает курятина. быстро готовится, а вкусно как, м-м-м! представляю рецепт бефстроганова из куриной грудки, думаю вам понравится. а с красивым оформлением пойдет и как горячее блюдо на новый год 2025.","[филе куриное (грудка) - 500 г, лук репчатый - 1 шт., мука - 1 ст. ложка, сливки - 150 мл, томатный сок - 50 мл, горчица - 1 ч. ложка, соль - 1 ч. ложка, перец - 1 щепотка, растительное масло - 3 ст. ложки]","[продукты для бефстроганова из курицы перед вами., как приготовить бефстроганов из куриной грудки:\n\r\nкуриную грудку вымыть и обсушить., филе куриной грудки разрезать на 2-3 плоских порционных куска и нарезать тонкой соломкой., разогреть сковороду, налить растительное масло. выложить мясо., филе посолить, поперчить и слегка обжарить 5-7 минут, помешивая., лук очистить и помыть, порезать полукольцами., затем к мясу добавить лук и обжарить до подрумянивания лука 4-5 минут, помешивая., добавить муку, хорошо перемешать., все залить сливками., смешать томатный сок с горчицей. хорошо перемешать., затем добавить к мясу горчицу и томатный сок., тушить куриный бефстроганов на слабом огне под крышкой 10-15 минут., бефстроганов из куриной грудки готов. подавать с любимым гарниром.\r\nприятного аппетита!]"



Очищенный датасет сохранен в '/content/recipes_texts_cleaned.csv'


In [4]:
import pandas as pd
from tqdm.auto import tqdm


cleaned_filename = '/content/recipes_texts_cleaned.csv'

chunks_filename = '/content/chunk1.parquet'


raw_df = pd.read_csv(cleaned_filename)


def safe_eval(x):
    if pd.isna(x):
        return []
    if isinstance(x, str):
        try:
            return eval(x)
        except:
            return []
    return x

raw_df['ingredients'] = raw_df['ingredients'].apply(safe_eval)
raw_df['recipe'] = raw_df['recipe'].apply(safe_eval)

chunk_idx = 0
chunk_dct = {"id": [], "chunk": [], "full_recipe": []}

for _, r in enumerate(tqdm(raw_df.iterrows(), total=raw_df.shape[0])):
    row = r[1]

    name = row["title"]
    full_recipe = [f"название рецепта: {name}"]

    description = row["description"]
    if pd.notna(description) and description != "nan":
        full_recipe.append(f"\nописание рецепта: {description}")

    ingredients = row["ingredients"]
    if ingredients and len(ingredients) > 0:
        full_recipe.append(
            "\nингредиенты:\n"
            + "\n".join([f"* {ingredient}" for ingredient in ingredients])
        )

    recipe = row["recipe"]
    if recipe and len(recipe) > 0:
        full_recipe.append(
            "\nпошаговый рецепт:\n"
            + "\n".join(
                [
                    f"шаг {step_idx + 1}: {step}"
                    for step_idx, step in enumerate(recipe)
                ]
            )
        )

    full_recipe_text = "\n".join(full_recipe)

    chunk_dct["id"].append(chunk_idx)
    chunk_dct["chunk"].append(full_recipe_text)
    chunk_dct["full_recipe"].append(full_recipe_text)

    chunk_idx += 1

chunk_df = pd.DataFrame(chunk_dct)
chunk_df.to_parquet(chunks_filename)

print(f"Файл {chunks_filename} успешно создан.")

  0%|          | 0/92144 [00:00<?, ?it/s]

Файл /content/chunk1.parquet успешно создан.


In [5]:
check_df = pd.read_parquet('/content/chunk1.parquet')
check_df.head(1)

,id,chunk,full_recipe
0,0,"название рецепта: ""бефстроганов"" из куриной гр...","название рецепта: ""бефстроганов"" из куриной гр..."


In [ ]:
import pandas as pd
import re
from tqdm.auto import tqdm


cleaned_filename = '/content/recipes_texts_cleaned.csv'
chunks_filename = '/content/chunk2.parquet'


raw_df = pd.read_csv(cleaned_filename)


chunk_idx = 0
chunk_list = []


for recipe_id, row in enumerate(tqdm(raw_df.iterrows(), total=raw_df.shape[0])):
    row = row[1]
    title = row["title"]
    description = row["description"]
    ingredients = row["ingredients"]
    recipe = row["recipe"]


    if pd.notna(ingredients):
        cleaned_ingredients = re.sub(r"[\[\]']", "", ingredients)
        cleaned_ingredients = re.sub(r",\s*", "\n", cleaned_ingredients)
        ingredients_list = [f"* {ing.strip()}" for ing in cleaned_ingredients.split('\n') if ing.strip()]
    else:
        ingredients_list = []


    if pd.notna(recipe):
        cleaned_recipe = re.sub(r"[\[\]']", "", recipe)
        cleaned_recipe = re.sub(r',\s*', ', ', cleaned_recipe)
        recipe_steps = re.split(r'\.,\s*', cleaned_recipe)
        recipe_steps = [step.strip() for step in recipe_steps if step.strip()]
    else:
        recipe_steps = []


    full_recipe = []
    if pd.notna(title):
        full_recipe.append(f"название рецепта: {title}")
    if pd.notna(description):
        full_recipe.append(f"\nописание рецепта: {description}")
    if ingredients_list:
        full_recipe.append("\nингредиенты:\n" + "\n".join(ingredients_list))
    if recipe_steps:
        full_recipe.append("\nпошаговый рецепт:\n" + "\n".join([f"шаг {i+1}: {step}" for i, step in enumerate(recipe_steps)]))
    full_recipe_text = "\n".join(full_recipe)


    if pd.notna(title):
        chunk_list.append({
            'chunk_id': chunk_idx,
            'recipe_id': recipe_id,
            'chunk_type': 'title',
            'chunk_text': title,
            'full_recipe': full_recipe_text
        })
        chunk_idx += 1


    if pd.notna(description):
        chunk_list.append({
            'chunk_id': chunk_idx,
            'recipe_id': recipe_id,
            'chunk_type': 'description',
            'chunk_text': description,
            'full_recipe': full_recipe_text
        })
        chunk_idx += 1


    if ingredients_list:
        ingredients_text = "\n".join(ingredients_list)
        chunk_list.append({
            'chunk_id': chunk_idx,
            'recipe_id': recipe_id,
            'chunk_type': 'ingredients',
            'chunk_text': ingredients_text,
            'full_recipe': full_recipe_text
        })
        chunk_idx += 1


    if recipe_steps:
        for step in recipe_steps:
            chunk_list.append({
                'chunk_id': chunk_idx,
                'recipe_id': recipe_id,
                'chunk_type': 'recipe_part',
                'chunk_text': step,
                'full_recipe': full_recipe_text
            })
            chunk_idx += 1


chunk_df = pd.DataFrame(chunk_list)
chunk_df.to_parquet(chunks_filename)

print(f"Файл {chunks_filename} успешно создан.")

  0%|          | 0/92144 [00:00<?, ?it/s]

In [18]:
chunk_df.head(20)

,chunk_id,recipe_id,chunk_type,chunk_text,full_recipe
0,0,0,title,"""бефстроганов"" из куриной грудки","название рецепта: ""бефстроганов"" из куриной грудки\n\nописание рецепта: фальшивый бефстроганов по мотивам любимого рецепта. меня всегда выручает курятина. быстро готовится, а вкусно как, м-м-м! представляю рецепт бефстроганова из куриной грудки, думаю вам понравится. а с красивым оформлением пойдет и как горячее блюдо на новый год 2025.\n\nингредиенты:\n* филе куриное (грудка) - 500 г\n* лук репчатый - 1 шт.\n* мука - 1 ст. ложка\n* сливки - 150 мл\n* томатный сок - 50 мл\n* горчица - 1 ч. ложка\n* соль - 1 ч. ложка\n* перец - 1 щепотка\n* растительное масло - 3 ст. ложки\n\nпошаговый рецепт:\nшаг 1: продукты для бефстроганова из курицы перед вами\nшаг 2: как приготовить бефстроганов из куриной грудки:\n\r\nкуриную грудку вымыть и обсушить\nшаг 3: филе куриной грудки разрезать на 2-3 плоских порционных куска и нарезать тонкой соломкой\nшаг 4: разогреть сковороду, налить растительное масло. выложить мясо\nшаг 5: филе посолить, поперчить и слегка обжарить 5-7 минут, помешивая\nшаг 6: лук очистить и помыть, порезать полукольцами\nшаг 7: затем к мясу добавить лук и обжарить до подрумянивания лука 4-5 минут, помешивая\nшаг 8: добавить муку, хорошо перемешать\nшаг 9: все залить сливками\nшаг 10: смешать томатный сок с горчицей. хорошо перемешать\nшаг 11: затем добавить к мясу горчицу и томатный сок\nшаг 12: тушить куриный бефстроганов на слабом огне под крышкой 10-15 минут\nшаг 13: бефстроганов из куриной грудки готов. подавать с любимым гарниром.\r\nприятного аппетита!"
1,1,0,description,"фальшивый бефстроганов по мотивам любимого рецепта. меня всегда выручает курятина. быстро готовится, а вкусно как, м-м-м! представляю рецепт бефстроганова из куриной грудки, думаю вам понравится. а с красивым оформлением пойдет и как горячее блюдо на новый год 2025.","название рецепта: ""бефстроганов"" из куриной грудки\n\nописание рецепта: фальшивый бефстроганов по мотивам любимого рецепта. меня всегда выручает курятина. быстро готовится, а вкусно как, м-м-м! представляю рецепт бефстроганова из куриной грудки, думаю вам понравится. а с красивым оформлением пойдет и как горячее блюдо на новый год 2025.\n\nингредиенты:\n* филе куриное (грудка) - 500 г\n* лук репчатый - 1 шт.\n* мука - 1 ст. ложка\n* сливки - 150 мл\n* томатный сок - 50 мл\n* горчица - 1 ч. ложка\n* соль - 1 ч. ложка\n* перец - 1 щепотка\n* растительное масло - 3 ст. ложки\n\nпошаговый рецепт:\nшаг 1: продукты для бефстроганова из курицы перед вами\nшаг 2: как приготовить бефстроганов из куриной грудки:\n\r\nкуриную грудку вымыть и обсушить\nшаг 3: филе куриной грудки разрезать на 2-3 плоских порционных куска и нарезать тонкой соломкой\nшаг 4: разогреть сковороду, налить растительное масло. выложить мясо\nшаг 5: филе посолить, поперчить и слегка обжарить 5-7 минут, помешивая\nшаг 6: лук очистить и помыть, порезать полукольцами\nшаг 7: затем к мясу добавить лук и обжарить до подрумянивания лука 4-5 минут, помешивая\nшаг 8: добавить муку, хорошо перемешать\nшаг 9: все залить сливками\nшаг 10: смешать томатный сок с горчицей. хорошо перемешать\nшаг 11: затем добавить к мясу горчицу и томатный сок\nшаг 12: тушить куриный бефстроганов на слабом огне под крышкой 10-15 минут\nшаг 13: бефстроганов из куриной грудки готов. подавать с любимым гарниром.\r\nприятного аппетита!"
2,2,0,ingredients,* филе куриное (грудка) - 500 г\n* лук репчатый - 1 шт.\n* мука - 1 ст. ложка\n* сливки - 150 мл\n* томатный сок - 50 мл\n* горчица - 1 ч. ложка\n* соль - 1 ч. ложка\n* перец - 1 щепотка\n* растительное масло - 3 ст. ложки,"название рецепта: ""бефстроганов"" из куриной грудки\n\nописание рецепта: фальшивый бефстроганов по мотивам любимого рецепта. меня всегда выручает курятина. быстро готовится, а вкусно как, м-м-м! представляю рецепт бефстроганова из куриной грудки, думаю вам понравится. а с красивым оформлением пойдет и как горячее блюдо на новый год 2025.\n\nингредиенты:\n* филе куриное (грудка) 

In [24]:
import pandas as pd
import re
from tqdm.auto import tqdm


cleaned_filename = '/content/recipes_texts_cleaned.csv'
chunks_filename = '/content/chunk3.parquet'


raw_df = pd.read_csv(cleaned_filename)


chunk_list = []
chunk_idx = 0


for recipe_id, row in enumerate(tqdm(raw_df.iterrows(), total=raw_df.shape[0])):
    row = row[1]
    title = row["title"]
    description = row["description"]
    ingredients = row["ingredients"]
    recipe = row["recipe"]


    if pd.notna(ingredients):
        cleaned_ingredients = re.sub(r"[\[\]']", "", ingredients)  # Убираем [], '
        cleaned_ingredients = re.sub(r",\s*", "\n", cleaned_ingredients)  # Запятые -> переносы строк
        ingredients_list = [f"* {ing.strip()}" for ing in cleaned_ingredients.split('\n') if ing.strip()]
        ingredients_text = "\n".join(ingredients_list)
    else:
        ingredients_text = ""


    if pd.notna(recipe):
        cleaned_recipe = re.sub(r"[\[\]']", "", recipe)  # Убираем [], '
        recipe_steps = re.split(r'\.,\s*', cleaned_recipe)  # Разбиваем по ".,"
        recipe_steps = [step.strip() for step in recipe_steps if step.strip()]
        recipe_text = "\n".join([f"шаг {i+1}: {step}" for i, step in enumerate(recipe_steps)])
        recipe_chunk_text = f"Название рецепта: {title}\nПошаговый рецепт:\n{recipe_text}"
    else:
        recipe_chunk_text = f"Название рецепта: {title}" if pd.notna(title) else ""


    full_recipe = []
    if pd.notna(title):
        full_recipe.append(f"название рецепта: {title}")
    if pd.notna(description):
        full_recipe.append(f"\nописание рецепта: {description}")
    if ingredients_list:
        full_recipe.append("\nингредиенты:\n" + "\n".join(ingredients_list))
    if recipe_steps:
        full_recipe.append("\nпошаговый рецепт:\n" + "\n".join([f"шаг {i+1}: {step}" for i, step in enumerate(recipe_steps)]))
    full_recipe_text = "\n".join(full_recipe)


    if recipe_chunk_text:
        chunk_list.append({
            'chunk_id': chunk_idx,
            'recipe_id': recipe_id,
            'chunk_type': 'recipe',
            'chunk_text': recipe_chunk_text,
            'full_recipe': full_recipe_text
        })
        chunk_idx += 1


    if ingredients_text:
        chunk_list.append({
            'chunk_id': chunk_idx,
            'recipe_id': recipe_id,
            'chunk_type': 'ingredients',
            'chunk_text': ingredients_text,
            'full_recipe': full_recipe_text
        })
        chunk_idx += 1


chunk_df = pd.DataFrame(chunk_list)
chunk_df.to_parquet(chunks_filename)

print(f"Файл {chunks_filename} успешно создан.")

  0%|          | 0/92144 [00:00<?, ?it/s]

Файл /content/chunk3.parquet успешно создан.


In [25]:
chunk_df.head(20)

,chunk_id,recipe_id,chunk_type,chunk_text,full_recipe
0,0,0,recipe,"Название рецепта: ""бефстроганов"" из куриной грудки\nПошаговый рецепт:\nшаг 1: продукты для бефстроганова из курицы перед вами\nшаг 2: как приготовить бефстроганов из куриной грудки:\n\r\nкуриную грудку вымыть и обсушить\nшаг 3: филе куриной грудки разрезать на 2-3 плоских порционных куска и нарезать тонкой соломкой\nшаг 4: разогреть сковороду, налить растительное масло. выложить мясо\nшаг 5: филе посолить, поперчить и слегка обжарить 5-7 минут, помешивая\nшаг 6: лук очистить и помыть, порезать полукольцами\nшаг 7: затем к мясу добавить лук и обжарить до подрумянивания лука 4-5 минут, помешивая\nшаг 8: добавить муку, хорошо перемешать\nшаг 9: все залить сливками\nшаг 10: смешать томатный сок с горчицей. хорошо перемешать\nшаг 11: затем добавить к мясу горчицу и томатный сок\nшаг 12: тушить куриный бефстроганов на слабом огне под крышкой 10-15 минут\nшаг 13: бефстроганов из куриной грудки готов. подавать с любимым гарниром.\r\nприятного аппетита!","название рецепта: ""бефстроганов"" из куриной грудки\n\nописание рецепта: фальшивый бефстроганов по мотивам любимого рецепта. меня всегда выручает курятина. быстро готовится, а вкусно как, м-м-м! представляю рецепт бефстроганова из куриной грудки, думаю вам понравится. а с красивым оформлением пойдет и как горячее блюдо на новый год 2025.\n\nингредиенты:\n* филе куриное (грудка) - 500 г\n* лук репчатый - 1 шт.\n* мука - 1 ст. ложка\n* сливки - 150 мл\n* томатный сок - 50 мл\n* горчица - 1 ч. ложка\n* соль - 1 ч. ложка\n* перец - 1 щепотка\n* растительное масло - 3 ст. ложки\n\nпошаговый рецепт:\nшаг 1: продукты для бефстроганова из курицы перед вами\nшаг 2: как приготовить бефстроганов из куриной грудки:\n\r\nкуриную грудку вымыть и обсушить\nшаг 3: филе куриной грудки разрезать на 2-3 плоских порционных куска и нарезать тонкой соломкой\nшаг 4: разогреть сковороду, налить растительное масло. выложить мясо\nшаг 5: филе посолить, поперчить и слегка обжарить 5-7 минут, помешивая\nшаг 6: лук очистить и помыть, порезать полукольцами\nшаг 7: затем к мясу добавить лук и обжарить до подрумянивания лука 4-5 минут, помешивая\nшаг 8: добавить муку, хорошо перемешать\nшаг 9: все залить сливками\nшаг 10: смешать томатный сок с горчицей. хорошо перемешать\nшаг 11: затем добавить к мясу горчицу и томатный сок\nшаг 12: тушить куриный бефстроганов на слабом огне под крышкой 10-15 минут\nшаг 13: бефстроганов из куриной грудки готов. подавать с любимым гарниром.\r\nприятного аппетита!"
1,1,0,ingredients,* филе куриное (грудка) - 500 г\n* лук репчатый - 1 шт.\n* мука - 1 ст. ложка\n* сливки - 150 мл\n* томатный сок - 50 мл\n* горчица - 1 ч. ложка\n* соль - 1 ч. ложка\n* перец - 1 щепотка\n* растительное масло - 3 ст. ложки,"название рецепта: ""бефстроганов"" из куриной грудки\n\nописание рецепта: фальшивый бефстроганов по мотивам любимого рецепта. меня всегда выручает курятина. быстро готовится, а вкусно как, м-м-м! представляю рецепт бефстроганова из куриной грудки, думаю вам понравится. а с красивым оформлением пойдет и как горячее блюдо на новый год 2025.\n\nингредиенты:\n* филе куриное (грудка) - 500 г\n* лук репчатый - 1 шт.\n* мука - 1 ст. ложка\n* сливки - 150 мл\n* томатный сок - 50 мл\n* горчица - 1 ч. ложка\n* соль - 1 ч. ложка\n* перец - 1 щепотка\n* растительное масло - 3 ст. ложки\n\nпошаговый рецепт:\nшаг 1: продукты для бефстроганова из курицы перед вами\nшаг 2: как приготовить бефстроганов из куриной грудки:\n\r\nкуриную грудку вымыть и обсушить\nшаг 3: филе куриной грудки разрезать на 2-3 плоских порционных куска и нарезать тонкой соломкой\nшаг 4: разогреть сковороду, налить растительное масло. выложить мясо\nшаг 5: филе посолить, поперчить и слегка обжарить 5-7 минут, помешивая\nшаг 6: лук очистить и помыть, порезать полукольцами\nшаг 7: затем к мясу добавить лук и обжарить до подрумянивания лука 4-5 минут, помешивая\nшаг 8: добавить муку, хорошо перемешать\nшаг 9: все залить сливками\nшаг 10: смешать томатный сок с г

In [6]:
!cp -r /content/chunk1.parquet /content/drive/MyDrive/
!cp -r /content/chunk2.parquet /content/drive/MyDrive/
!cp -r /content/chunk3.parquet /content/drive/MyDrive/